Sascha Spors,
Professorship Signal Theory and Digital Signal Processing,
Institute of Communications Engineering (INT),
Faculty of Computer Science and Electrical Engineering (IEF),
University of Rostock,
Germany

# Data Driven Audio Signal Processing - A Tutorial with Computational Examples

Winter Semester 2021/22 (Master Course #24512)

- lecture: https://github.com/spatialaudio/data-driven-audio-signal-processing-lecture
- tutorial: https://github.com/spatialaudio/data-driven-audio-signal-processing-exercise

Feel free to contact lecturer frank.schultz@uni-rostock.de

# Exercise 7: Least Squares Solution / Left Inverse in SVD and QR Domain

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import svd, diagsvd, qr, inv, pinv, norm
from numpy.linalg import matrix_rank
np.set_printoptions(precision=2, floatmode='fixed', suppress=True)

matplotlib_widget_flag = False

For the given full-column rank matrix $\mathbf{A}$ (tall/thin shape with independent columns) and  outcome vector $\mathbf{b}$ the linear set of equations

$\mathbf{A} \mathbf{x} = \mathbf{b}$

is to be solved for unknowns $\mathbf{x}$. We obviously cannot invert $\mathbf{A}$, so we must find a best possible row space estimate $\hat{\mathbf{x}}$.

### Least Squares Solution

great material, strong recommendation:
- Gilbert Strang (2020): "Linear Algebra for Everyone", Wellesley-Cambridge Press, Ch. 4.3
- Gilbert Strang (2019): "Linear Algebra and Learning from Data", Wellesley-Cambridge Press, Ch. II.2

We know for sure, that pure row space $\hat{\mathbf{x}}$ maps to pure column space $\hat{\mathbf{b}}$

$\mathbf{A} \hat{\mathbf{x}} = \hat{\mathbf{b}}$

The given outcome vector $\mathbf{b}$ might not necessarily (in practical problems probably never!) live in pure column space of $\mathbf{A}$, we therefore need an offset (error) vector to get there

$\mathbf{A} \hat{\mathbf{x}} + \mathbf{e} = \mathbf{b}$

We want to find the one $\hat{\mathbf{x}}$ that yields the smallest $||\mathbf{e}||_2^2$ or equivalently $||\mathbf{e}||_2$. This is basically our optimization criterion, known as least squares.

So, thinking in vector addition

$\mathbf{b} = \hat{\mathbf{b}} + \mathbf{e} \rightarrow \mathbf{e} = \mathbf{b} - \hat{\mathbf{b}}$

we can geometrically figure (imagine this is in 2D / 3D) that smallest $||\mathbf{e}||_2^2$ is achieved when we span a **right-angled triangle** using $\mathbf{b}$ as hypotenuse.

Therefore, $\hat{\mathbf{b}} \perp \mathbf{e}$. This means that $\mathbf{e}$ must live in left null space of $\mathbf{A}$, which is perpendicular to the column space where $\hat{\mathbf{b}}$ lives.

Left null space requirement formally written as $\mathbf{A}^\mathrm{H} \mathbf{e} = \mathbf{0}$ can be utilized as

$\mathbf{A} \hat{\mathbf{x}} + \mathbf{e} = \mathbf{b} \rightarrow \mathbf{A}^\mathrm{H}\mathbf{A} \hat{\mathbf{x}} + \mathbf{A}^\mathrm{H}\mathbf{e} = \mathbf{A}^\mathrm{H}\mathbf{b} \rightarrow \mathbf{A}^\mathrm{H}\mathbf{A} \hat{\mathbf{x}} = \mathbf{A}^\mathrm{H}\mathbf{b}$

The last equation in the line is known as normal equation.

This can be solved using the left inverse of $\mathbf{A}^\mathrm{H} \mathbf{A}$ (this matrix is full rank and therefore invertible)

$(\mathbf{A}^\mathrm{H} \mathbf{A})^{-1} (\mathbf{A}^\mathrm{H} \mathbf{A}) \hat{\mathbf{x}} = (\mathbf{A}^\mathrm{H} \mathbf{A})^{-1} \mathbf{A}^\mathrm{H} \mathbf{b}$

Since for left inverse $(\mathbf{A}^\mathrm{H} \mathbf{A})^{-1} (\mathbf{A}^\mathrm{H} \mathbf{A}) = \mathbf{I}$ holds, we get the least-squares sense solution for $\mathbf{x}$ in the row space of $\mathbf{A}$

$\hat{\mathbf{x}} = (\mathbf{A}^\mathrm{H} \mathbf{A})^{-1} \mathbf{A}^\mathrm{H} \mathbf{b}$

We find the **left inverse** of $\mathbf{A}$ as

$\mathbf{A}^{+L} = (\mathbf{A}^\mathrm{H} \mathbf{A})^{-1} \mathbf{A}^\mathrm{H}$

### Least Squares Solution in SVD Domain

great material, strong recommendation:
- Gilbert Strang (2019): "Linear Algebra and Learning from Data", Wellesley-Cambridge Press, p.125ff

The left inverse 

$\mathbf{A}^{+L} = (\mathbf{A}^\mathrm{H} \mathbf{A})^{-1} \mathbf{A}^\mathrm{H}$

in terms of SVD

$\mathbf{A}^{+L} = ((\mathbf{U}\mathbf{S}\mathbf{V}^\mathrm{H})^\mathrm{H} \mathbf{U}\mathbf{S}\mathbf{V}^\mathrm{H})^{-1} (\mathbf{U}\mathbf{S}\mathbf{V}^\mathrm{H})^\mathrm{H}$

$\mathbf{A}^{+L} = (\mathbf{V}\mathbf{S}^\mathrm{H}\mathbf{S}\mathbf{V}^\mathrm{H})^{-1} (\mathbf{V}\mathbf{S}^\mathrm{H}\mathbf{U}^\mathrm{H})$

$\mathbf{A}^{+L} = \mathbf{V} (\mathbf{S}^\mathrm{H}\mathbf{S})^{-1} \mathbf{V}^\mathrm{H} \mathbf{V}\mathbf{S}^\mathrm{H}\mathbf{U}^\mathrm{H}$

$\mathbf{A}^{+L} = \mathbf{V} (\mathbf{S}^\mathrm{H}\mathbf{S})^{-1} \mathbf{S}^\mathrm{H}\mathbf{U}^\mathrm{H}$

$\mathbf{A}^{+L} = \mathbf{V} \mathbf{S}^\mathrm{+L} \mathbf{U}^\mathrm{H}$

allows for a convenient discussion, how singular values act when mapping column space back to row space.

Considering only one $\sigma_i$ and corresponding left/right singular vectors, the left inverse $\mathbf{S}^\mathrm{+L} = (\mathbf{S}^\mathrm{H}\mathbf{S})^{-1} \mathbf{S}^\mathrm{H}$ reduces to

$\frac{\sigma_i}{\sigma_i^2} = \frac{1}{\sigma_i}$

For very, very small $\sigma_i$, the inversion thus leads to huge values, which might be not meaningful as this(these) weighted $\mathbf{v}_i$ vector(s) then dominate(s) the row space solution. Small changes in $\sigma_i$ then lead to comparably large changes in the row space solution $\hat{\mathbf{x}}$. So-called ridge regression (aka Tikhonov regularization) is a straightforward workaround for ill-conditioned matrices. See stuff below.

### Least Squares Solution in QR Domain

great material, strong recommendation:
- Gilbert Strang (2020): "Linear Algebra for Everyone", Wellesley-Cambridge Press, p.170ff
- Gilbert Strang (2019): "Linear Algebra and Learning from Data", Wellesley-Cambridge Press, p.128ff

The normal equation

$\mathbf{A}^\mathrm{H}\mathbf{A} \hat{\mathbf{x}} = \mathbf{A}^\mathrm{H}\mathbf{b}$

can be conveniently given as QR decomposition (recall $\mathbf{Q}^\mathrm{H} \mathbf{Q}=\mathbf{I}$ due to Gram-Schmidt orthonormalization)

$(\mathbf{Q R})^\mathrm{H}\mathbf{Q R} \hat{\mathbf{x}} = (\mathbf{Q R})^\mathrm{H}\mathbf{b}$

$\mathbf{R}^\mathrm{H} \mathbf{Q}^\mathrm{H} \mathbf{Q R} \hat{\mathbf{x}} = (\mathbf{Q R})^\mathrm{H}\mathbf{b}$

$\mathbf{R}^\mathrm{H} \mathbf{R} \hat{\mathbf{x}} = \mathbf{R}^\mathrm{H} \mathbf{Q}^\mathrm{H} \mathbf{b}$

$\mathbf{R} \hat{\mathbf{x}} = \mathbf{Q}^\mathrm{H} \mathbf{b}$

Numerical approaches try to solve for $\hat{\mathbf{x}}$ using the last line by back substitution, given that we found a numerically robust solution for upper triangle $\mathbf{R}$ and orthonormal $\mathbf{Q}$.

We should not expect that algorithms solve

$\hat{\mathbf{x}} = \mathbf{R}^{+L} \mathbf{Q}^\mathrm{H} \mathbf{b}$

with the left inverse $\mathbf{R}^{+L}$ of upper triangle $\mathbf{R}$, we should not do this for non-toy-examples as well.

In [ ]:
rng = np.random.default_rng(1)
mean, stdev = 0, 0.01
M = 100
N = 3
A = rng.normal(mean, stdev, [M, N])
print('rank =', matrix_rank(A), '== number of cols =', N)

In [ ]:
if matplotlib_widget_flag:
    %matplotlib widget

In [ ]:
[Q, R] = qr(A)
[U, s, Vh] = svd(A)
print('sing vals', s)
V = Vh.conj().T

# scipy function
Ali_pinv = pinv(A)

# manual normal equation solver
Ali_man = inv(A.conj().T @ A) @ A.conj().T

# SVD
Si = diagsvd(1/s, N, M)  # works if array s has only non-zero entries
Ali_svd = V @ Si @ U.conj().T

# QR
Ali_qr = pinv(R) @ Q.conj().T

print('pinv == inverse via normal eq?', np.allclose(Ali_pinv, Ali_man))
print('pinv == inverse via SVD?', np.allclose(Ali_pinv, Ali_svd))
print('pinv == inverse via QR?', np.allclose(Ali_pinv, Ali_qr))

In [ ]:
# create b from one column space entry and one left null space entry
# note that we use unit length vectors for convenience: ||e||_2^2 = 1
bh = U[:,0]  # choose one of col space
e = U[:,N]  # assuming rank N -> we choose some one of left null space
b = bh + e

# find xh in row space
xh = Ali_pinv @ b  # only bh gets mapped to row space (this is our LS solution xh), e is mapped to zero vec
print(xh)

In [ ]:
print('norm(A @ xh - bh, 2) == 0 -> ', norm(A @ xh - bh, 2))  # == 0

print('||e||_2^2:')
print(norm(A @ xh - b, 2)**2)
print(norm(e, 2)**2)

### Ridge Regression / Regularization in SVD Domain

For ridge regression, aka Tikhonov regularization, aka regression with penalty on $||\mathbf{x}||_2^2$ the ridge, left inverse

$\mathbf{A}^{+\mathrm{L,Ridge}} = \mathbf{V} \left((\mathbf{S}^\mathrm{H}\mathbf{S} + \beta^2 \mathbf{I})^{-1} \mathbf{S}^\mathrm{H}\right) \mathbf{U}^\mathrm{H}$

yields the solution

$\hat{\mathbf{x}}^\mathrm{Ridge} = \mathbf{A}^{+\mathrm{L,Ridge}}  \mathbf{b}$

for the minimization problem

$\mathrm{min}_\mathbf{x} \left(||\mathbf{A} \mathbf{x} - \mathbf{b}||_2^2 + \beta^2 ||\mathbf{x}||_2^2 \right)$.

For limit $\beta^2=0$ this is identical to above standard least squares solution.

For a single singular value the ridge penalty becomes
$\frac{\sigma_i}{\sigma_i^2 + \beta^2}$, which can be discussed conveniently with below plot.

In [ ]:
beta = 1/10
lmb = beta**2

singval = np.logspace(-4, 4, 2**5)
# ridge regression
inv_singval = singval / (singval**2 + beta**2)

plt.plot(singval, 1 / singval, label='no penalty')
plt.plot(singval, inv_singval, label='penalty')
plt.xscale('log')
plt.yscale('log')
plt.xticks(10.**np.arange(-4,5))
plt.yticks(10.**np.arange(-4,5))
plt.xlabel(r'$\sigma_i$')
plt.ylabel(r'$\sigma_i \,\,\,/\,\,\, (\sigma_i^2 + \beta^2)$')
plt.title(r'ridge penalty, $\beta =$'+str(beta))
plt.legend()
plt.grid()
plt.axis('equal')
plt.show()
print('beta =', beta, 'beta^2 = lambda =', lmb)

In [ ]:
rng = np.random.default_rng(1)
mean, stdev = 0, 10
M, N = 3, 3
A_tmp = rng.normal(mean, stdev, [M, N])
[U_tmp, s_tmp, Vh_tmp] = svd(A_tmp)
V_tmp = Vh_tmp.conj().T
s_tmp = [10, 8, 0.5]  # create sing vals
S_tmp = diagsvd(s_tmp, M, N)

# create full rank square matrix to work with (no nullspaces except 0-vectors!)
A = U_tmp @ S_tmp @ Vh_tmp
[U, s, Vh] = svd(A)
print('A\n', A)
print('rank of A: ', matrix_rank(A))
print('sigma', s)
S = diagsvd(s, M, N)
V = Vh.conj().T

# b as column space linear combination
b = 1*U[:,0] + 1*U[:,1] + 1*U[:,2]

xh = inv(A) @ b
print('xh =', xh, '\nA xh =', A @ xh, '\nb =', b)
print('inverted sigma no penalty: ', 1 / s)  # == (because in b all U weighted with unity gain)
print('||xh||_2^2 =', norm(xh,2))
print('norm of vec: inverted sigma no penalty: ', norm(1 / s,2))

lmb = 2
Sli_ridge = inv(S.conj().T @ S + lmb*np.eye(3)) @ S.conj().T
Ali_ridge = V @ Sli_ridge @ U.conj().T
xh_ridge = Ali_ridge @ b
print('xh_ridge =', xh_ridge, '\nA xh_ridge =', A @ xh_ridge, '\nb = ', b)
print('inverted sigma with penalty: ', s / (s**2 + lmb))  # == (because in b all U weighted with unity gain)
print('||xh_ridge||_2^2 =', norm(xh_ridge,2))
print('norm of vec: inverted sigma with penalty: ', norm(s / (s**2 + lmb),2))

fig1 = plt.figure(figsize=(5,5))
ax = plt.axes(projection='3d')
w = Vh @ xh
wr = Vh @ xh_ridge
for n in range(3):
    ax.plot([0, w[n]*V[0,n]], [0, w[n]*V[1,n]], [0, w[n]*V[2,n]], color='C'+str(n), lw=1, ls=':', label=r'$\hat{x}$@$v_i$, no penalty')
    ax.plot([0, wr[n]*V[0,n]], [0, wr[n]*V[1,n]], [0, wr[n]*V[2,n]], color='C'+str(n), lw=3, ls='-', label=r'$\hat{x}$@$v_i$, penalty')
ax.plot([0, xh[0]], [0, xh[1]], [0, xh[2]], 'black', label=r'$\hat{x}$, no penalty')
ax.plot([0, xh_ridge[0]], [0, xh_ridge[1]], [0, xh_ridge[2]], 'C7', label='$\hat{x}$, penalty')
ax.set_xlabel(r'$x$')
ax.set_ylabel(r'$y$')
ax.set_zlabel(r'$z$')
lim = 1
ax.set_xlim(-lim, lim)
ax.set_ylim(-lim, lim)
ax.set_zlim(-lim, lim)
ax.set_title('V / row space')
plt.legend()

fig2 = plt.figure(figsize=(5,5))
ax = plt.axes(projection='3d')
w = Vh @ xh
wr = Vh @ xh_ridge
for n in range(3):
    ax.plot([0, U[0,n]], [0, U[1,n]], [0, U[2,n]], color='C'+str(n), lw=2, ls='-', label=r'$u_i$')
    ax.plot([0, s[n]*U[0,n]], [0, s[n]*U[1,n]], [0, s[n]*U[2,n]], color='C'+str(n), lw=1, ls=':', label=r'$\sigma_i \cdot u_i$')
ax.plot([0, b[0]], [0, b[1]], [0, b[2]], 'black', lw=1, label=r'$b$')
ax.set_xlabel(r'$x$')
ax.set_ylabel(r'$y$')
ax.set_zlabel(r'$z$')
lim = 5
ax.set_xlim(-lim, lim)
ax.set_ylim(-lim, lim)
ax.set_zlim(-lim, lim)
ax.set_title('U / row space')
plt.legend();

In [ ]:
if matplotlib_widget_flag:
    plt.close(fig1)
    plt.close(fig2)

## Copyright

- the notebooks are provided as [Open Educational Resources](https://en.wikipedia.org/wiki/Open_educational_resources)
- feel free to use the notebooks for your own purposes
- the text is licensed under [Creative Commons Attribution 4.0](https://creativecommons.org/licenses/by/4.0/)
- the code of the IPython examples is licensed under under the [MIT license](https://opensource.org/licenses/MIT)
- please attribute the work as follows: *Frank Schultz, Data Driven Audio Signal Processing - A Tutorial Featuring Computational Examples, University of Rostock* ideally with relevant file(s), github URL https://github.com/spatialaudio/data-driven-audio-signal-processing-exercise, commit number and/or version tag, year.